In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from scipy import stats

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
titanic_train = pd.read_csv('train.csv')
titanic_test = pd.read_csv('test.csv')

In [ ]:
# We have a train dataset, which we'll use to train and validate our models, and the test dataset, which we'll submit the values
# Printing the columns and number of instances on the train and test datasets
print(titanic_train.columns)
print(titanic_test.columns)
print(titanic_train.shape)
print(titanic_test.shape)

In [ ]:
# Counting the number of elements in each attribute
print(titanic_train.count())
# Checking the type of each attribute
print(titanic_train.dtypes)

In [ ]:
# Using describe to analyse the numerial attributes
titanic_train.describe()
# There's missing values in Age and Cabin;
# 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked' are strings, we need to check if they're categorial or nominal;

In [ ]:
print(titanic_train.Name.head())
# Name is identifier - discard
print(titanic_train.Sex.head())
# Sex is categorical, but binary - change to 0/1
print(titanic_train.Ticket.head())
# Ticket is a string, description - discard
print(titanic_train.Cabin.head())
# Cabin is a identifier, might be useful - discard, however
print(titanic_train.Embarked.head())
# Embarked is nominal - one-hot needed, discard for now

In [ ]:
# Create copy
ttn = titanic_train
# Drop all attributes we don't want
ttn = ttn.drop('PassengerId', axis=1)
ttn = ttn.drop('Name', axis=1)
ttn = ttn.drop('Ticket', axis=1)
ttn = ttn.drop('Cabin', axis=1)
ttn = ttn.drop('Embarked', axis=1)
# Change sex to a integer value
ttn['Sex'] = ttn['Sex'].replace({'male': 0, 'female': 1})
ttn_na = ttn.copy()
# Fill in age with the mean
ttn['Age'] = ttn['Age'].fillna(ttn['Age'].mean())

In [ ]:
# Check correlation
for i in ttn.columns:
    print(ttn.corr()[i])

In [ ]:
# Normalize the dataset
temp = ttn.copy()
for feature_name in ttn.columns:
    maxv = ttn[feature_name].max()
    minv = ttn[feature_name].min()
    temp[feature_name] = (ttn[feature_name] - minv) / (maxv - minv)
ttn_norm = temp

In [ ]:
# Function to split train (80%) and test (20%)
def split(df):
    X_train, X_test = train_test_split(df, test_size = 0.2)
    Y_train = X_train['Survived']
    X_train = X_train.drop('Survived', axis=1)
    Y_test = X_test['Survived']
    X_test = X_test.drop('Survived', axis=1)
    return X_train, Y_train, X_test, Y_test

# Split features and class for the cross validation
def crossval_split(df):
    Y = df['Survived']
    X = df.drop('Survived', axis=1)
    return X, Y

#K-NN function
def knn_run(db, n=5, weights='uniform', algorithm='auto', p=2):    
    X, Y = crossval_split(db)
    clf = KNeighborsClassifier(n_neighbors=n, weights=weights, algorithm=algorithm, p=p)
    scores = cross_val_score(clf, X, Y, cv=10)
    return scores

# Gaussian Naive Bayes
def bayes_run(db):
    X, Y = crossval_split(db)
    clf = GaussianNB()
    scores = cross_val_score(clf, X, Y, cv=10)
    return scores

# Decision Tree
def dectree_run(db, depth=None):
    X, Y = crossval_split(db)
    clf = tree.DecisionTreeClassifier(max_depth=depth)    
    scores = cross_val_score(clf, X, Y, cv=10)
    return scores

# SVM
from sklearn import svm
def svm_run(db, kernel='rbf', gamma='scale'):
    X, Y = crossval_split(db)
    clf = svm.SVC(gamma=gamma, kernel=kernel)
    scores = cross_val_score(clf, X, Y, cv=10)
    return scores

In [ ]:
knn_score_list_orig = []
knn_score_list = []
for i in range(1,11):
    knn_score_list_orig.append(knn_run(ttn, n=i))
for i in range(1,11):
    knn_score_list.append(knn_run(ttn_norm, n=i))
    
# Compare normal database with a normalized one
fig1, ax1 = plt.subplots(ncols=2, sharey='all')
ax1[0].boxplot(knn_score_list_orig)
ax1[1].boxplot(knn_score_list)
plt.show()

In [ ]:
# Compare knn/algorithms
knn_score_list_ball = []
knn_score_list_kd = []
knn_score_list_brute = []
for i in range(1,11):
    knn_score_list_ball.append(knn_run(ttn_norm, n=i, algorithm='ball_tree'))
for i in range(1,11):
    knn_score_list_kd.append(knn_run(ttn_norm, n=i, algorithm='kd_tree'))
for i in range(1,11):
    knn_score_list_brute.append(knn_run(ttn_norm, n=i, algorithm='brute'))

fig1, ax1 = plt.subplots(ncols=3, sharey='all')
ax1[0].boxplot(knn_score_list_ball)
ax1[1].boxplot(knn_score_list_kd)
ax1[2].boxplot(knn_score_list_brute)
plt.show()

In [ ]:
# Compare knn/weights
knn_score_list_uniform = []
knn_score_list_distance = []
for i in range(1,11):
    knn_score_list_uniform.append(knn_run(ttn_norm, n=i, weights='uniform'))
for i in range(1,11):
    knn_score_list_distance.append(knn_run(ttn_norm, n=i, weights='distance'))

fig1, ax1 = plt.subplots(ncols=2, sharey='all')
ax1[0].boxplot(knn_score_list_uniform)
ax1[1].boxplot(knn_score_list_distance)
plt.show()

In [ ]:
# Naive Bayes
fig1, ax1 = plt.subplots()
ax1.boxplot([bayes_run(ttn), bayes_run(ttn_norm)])
plt.show()

In [ ]:
# Run Decision Tree to both datasets
dectree_score_orig = dectree_run(ttn) 
dectree_score = dectree_run(ttn_norm)
fig1, ax1 = plt.subplots()
ax1.boxplot([dectree_score_orig, dectree_score])
plt.show()

In [ ]:
# Test multiple depths
dectree_score_depth = []
for i in range(1,30):
    dectree_score_depth.append(dectree_run(ttn, depth=i))
fig1, ax1 = plt.subplots()
ax1.boxplot(dectree_score_depth)
plt.show()

In [ ]:
#
svm_score_orig = []
svm_score = []
for i in ['rbf', 'poly', 'sigmoid']:
    svm_score_orig.append(svm_run(ttn, kernel=i))
    svm_score.append(svm_run(ttn_norm, kernel=i))

fig1, ax1 = plt.subplots(ncols=2, sharey='all')
ax1[0].boxplot(svm_score_orig)
ax1[1].boxplot(svm_score)
plt.show()

In [ ]:
# Gridsearch test
X_train, Y_train, X_test, Y_test = split(ttn_norm)
param = {'kernel':['rbf', 'linear', ]}
svc = svm.SVC(gamma='scale')
clf = GridSearchCV(svc, param, cv=5)
clf.fit(X_train, Y_train)
print(clf.best_params_)
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['params'])

In [ ]:
# Compare everybody
fig1, ax1 = plt.subplots(ncols=4, sharey='all')
ax1[0].boxplot(knn_score_list_uniform)
ax1[1].boxplot([bayes_run(ttn), bayes_run(ttn_norm)])
ax1[2].boxplot(dectree_score_depth[0:10])
ax1[3].boxplot(svm_score)
plt.show()

In [ ]:
# Create copy
test = titanic_test
# Drop all attributes we don't want
test = test.drop('PassengerId', axis=1)
test = test.drop('Name', axis=1)
test = test.drop('Ticket', axis=1)
test = test.drop('Cabin', axis=1)
test = test.drop('Embarked', axis=1)
# Change sex to a integer value
test['Sex'] = test['Sex'].replace({'male': 0, 'female': 1})
# Fill in age with the mean
test = test.fillna(test.median())

# Normalize the dataset
temp = test.copy()
for feature_name in test.columns:
    maxv = test[feature_name].max()
    minv = test[feature_name].min()
    temp[feature_name] = (test[feature_name] - minv) / (maxv - minv)
test = temp

X, Y = crossval_split(ttn_norm)
clf = KNeighborsClassifier(n_neighbors=4)
clf.fit(X, Y)
prediction = clf.predict(test)

In [ ]:
# Export results
data = {'PassengerId': titanic_test['PassengerId'], 'Survived': prediction.astype(int)}
pd.DataFrame(data).to_csv('submission.csv', index=False)